# Available Data Types in NumPy

## Numerical Types: int, bool, float, and complex
Since most of your data science and numerical calculations will tend to involve numbers, they seem like the best place to start. There are essentially four numerical types in NumPy code, and each one can take a few different sizes.

The table below breaks down the details of these types:

|Name|# of Bits|Python Type|NumPy Type|
|:--|:--|:--|:--|
|Integer|64|int|np.int_|
|Booleans|8|bool|np.bool_|
|Float|64|float|np.float_|
|Complex|128|complex|np.complex_|

These are just the types that map to existing Python types. NumPy also has types for the smaller-sized versions of each, like 8-, 16-, and 32-bit integers, 32-bit single-precision floating-point numbers, and 64-bit single-precision complex numbers. The documentation lists them in their entirety.

To specify the type when creating an array, you can provide a `dtype` argument:

In [2]:
import numpy as np

In [8]:
np.array([1, 3, 5.5, 7.7, 9.2], dtype=np.single)

array([1. , 3. , 5.5, 7.7, 9.2], dtype=float32)

In [11]:
np.array([1, 3, 5.5, 7.7, 9.2], dtype=np.uint8)

array([1, 3, 5, 7, 9], dtype=uint8)

NumPy automatically converts your platform-independent type `np.single` to whatever fixed-size type your platform supports for that size. In this case, it uses `np.float32`. If your provided values don’t match the shape of the `dtype` you provided, then NumPy will either fix it for you or raise an error.

## String Types: Sized Unicode

Strings behave a little strangely in NumPy code because NumPy needs to know how many bytes to expect, which isn’t usually a factor in Python programming. Luckily, NumPy does a pretty good job at taking care of less complex cases for you:

In [14]:
import numpy as np

In [22]:
names = np.array(["bob", "amy", "han"], dtype=str)
names

array(['bob', 'amy', 'han'], dtype='<U3')

In [16]:
names.itemsize

12

In [18]:
names = np.array(["bob", "amy", "han"])
names

array(['bob', 'amy', 'han'], dtype='<U3')

In [19]:
more_names = np.array(["bobo", "jehosephat"])

In [20]:
more_names.dtype, more_names.itemsize

(dtype('<U10'), 40)

In [21]:
np.concatenate((names, more_names))

array(['bob', 'amy', 'han', 'bobo', 'jehosephat'], dtype='<U10')

In `names`, you provide a `dtype` of Python’s built-in `str` type, but in its output, it’s been converted into a little-endian Unicode string of size 3. When you check the size of a given item in input 4, you see that they’re each 12 bytes: three 4-byte Unicode characters.

> **Note:** When dealing with NumPy data types, you have to think about things like the endianness of your values. In this case, the dtype `'<U3'` means that each value is the size of three Unicode characters, with the least-significant byte stored first in memory and the most-significant byte stored last. A `dtype` of `'>U3'` would signify the reverse.
>
>As an example, NumPy represents the Unicode character “🐍” with the bytes `0xF4 0x01 0x00` with a dtype of `'<U1'` and `0x00 0x01 0xF4` with a `dtype` of `'>U1'`. Try it out by creating an array full of emoji, setting the dtype to one or the other, and then calling `.tobytes()` on your array!
>
>If you’d like to study up on how Python treats the ones and zeros of your normal Python data types, then the official documentation for the [struct library](https://docs.python.org/3/library/struct.html#struct-alignment), which is a standard library module that works with raw bytes, is another good resource.

When you combine that with an array that has a larger item to create a new array in input 8, NumPy helpfully figures out how big the new array’s items need to be and grows them all to size `<U10`.

But here’s what happens when you try to modify one of the slots with a value larger than the capacity of the dtype:

In [25]:
names[2] = "jamima"

names

array(['aaa', 'amy', 'jam'], dtype='<U3')

It doesn’t work as expected and truncates your value instead. If you already have an array, then NumPy’s automatic size detection won’t work for you. You get three characters and that’s it. The rest get lost in the void.

This is all to say that, in general, NumPy has your back when you’re working with strings, but you should always keep an eye on the size of your elements and make sure you have enough space when modifying or changing arrays in place.

## Structured Arrays

Originally, you learned that array items all have to be the same data type, but that wasn’t entirely correct. NumPy has a special kind of array, called a **record array** or **structured array**, with which you can specify a type and, optionally, a name on a per-column basis. This makes sorting and filtering even more powerful, and it can feel similar to working with data in Excel, CSVs, or relational databases.

Structured arrays or record arrays are useful when you perform computations, and at the same time you could keep closely related data together. For example, when you process incident data and each incident contains geographic coordinates and the occurrence time, while you calculate the final result, you can easily find the associated geographic locations and timepoint for further visualization. NumPy also provides powerful capabilities to create arrays of records, as multiple data types live in one NumPy array. However, one principle in NumPy that still needs to be honored is that the data type in each field (you can think of this as a column in the records) needs to be homogeneous. Here are some simple examples that show you how it works:

In [39]:
import numpy as np

In [40]:
x = np.empty((2,), dtype = ('i4, f4, a10')) 

In [41]:
x[:] = [(1, 0.5, 'NumPy'), (10, -0.5, 'Essential')] 

In [42]:
x

array([( 1,  0.5, b'NumPy'), (10, -0.5, b'Essential')],
      dtype=[('f0', '<i4'), ('f1', '<f4'), ('f2', 'S10')])

In the previous example, we created a one-dimensional record array using `numpy.empty()` and specified the data types for the elements-the first element is `i4` (a 32-bit integer, where `i` stands for a signed integer, and 4 means 4 bytes, like `np.int32`), the second element is a 32-bit float (`f` stands for float and also 4 bytes), and the third element is a string of length less than or equal to 10. We assign the values to the defined array following the data type orders we specified.

You can see the print-out of `x`, which now contains three different types of records, and we also get a default field name in dtype: `f0`, `f1`, and `f2`. Of course, you may specify your field names, as we'll show you in the following examples.

One thing to note here is that we used the print-out data type-there is a `<` in front of `i4` and `f4`, and `<` stands for byteorder **big-endian** (indicating the memory address increase order):

In [43]:
x[0]

(1, 0.5, b'NumPy')

In [46]:
x['f2']

array([b'NumPy', b'Essential'], dtype='|S10')

The way we retrieve data remains the same, we use the index to obtain the record, but moreover, we can use the field name to obtain the value of certain fields, so in the previous example, we used `f2` to obtain the string field. In the following example, we are going to create a view of `x`, named `y`, and see how it interacts with the original record array:

In [47]:
y = x['f0']

In [48]:
y

array([ 1, 10], dtype=int32)

In [49]:
y.strides

(18,)

In [50]:
y[:] = y + 0.5

In [51]:
y

array([ 1, 10], dtype=int32)

In [52]:
x

array([( 1,  0.5, b'NumPy'), (10, -0.5, b'Essential')],
      dtype=[('f0', '<i4'), ('f1', '<f4'), ('f2', 'S10')])

Here, `y` is the view of field `f0` in `x`. In the record arrays, the characteristics of NumPy arrays still remain. When you multiply the scalar 10, it still applies to whole array of `y` (the broadcasting rule), and it always honors the data type. You can see after the multiplication, we add `0.5` to `y`, but since the data type of field `f0` is a 32-bit integer, the result is still `[10, 100]`. Also, `y` is a view of `f0` in `x`, so they share the same memory block. When we print out `x` after the calculation in `y`, we can find that the values in `x` have also changed.

Before we go further into the record arrays, let's sort out how to define a record array. The easiest way is as shown in the previous example, where we initialize a NumPy array and use the string argument to specify the data type of fields.

There are many forms of string argument NumPy can accept (see https://numpy.org/doc/stable/user/basics.rec.html for details); the most preferred can be chosen from one of these:


|Data types | Representation|
|:--|:--|
|`b1` | Bytes|
|`i1,i2,i4,i8` | Signed integers with 1, 2, 4, and 8 bytes corresponding to them|
|`u1,u2,u4,u8` | Unsigned integers with 1, 2, 4, and 8 bytes|
|`f2,f4,f8` | Floats with 2, 4, and 8 bytes|
|`c8,c16` | Complex with 8 and 16 bytes|
|`a<n>` | Fixed length strings of length n|

You may also prefix the string arguments with a repeated number or a shape to define the dimension of the field, but it's still considered as just one field in the record arrays. Let's try using the shape as prefix to the string arguments in the following example:

In [62]:
z = np.ones((2,), dtype = ('3i4, (2,3)f4'))

In [63]:
z

array([([1, 1, 1], [[1., 1., 1.], [1., 1., 1.]]),
       ([1, 1, 1], [[1., 1., 1.], [1., 1., 1.]])],
      dtype=[('f0', '<i4', (3,)), ('f1', '<f4', (2, 3))])

In the previous example, field `f0` is a one-dimensional array with size `3` and `f1` is a two-dimensional array with shape `(2, 3)`. Now, we are clear about the structure of a record array and how to define it. You might be wondering whether the default field name can be changed to something meaningful in your analysis? Of course it can! This is how:

In [59]:
x.dtype.names 

('f0', 'f1', 'f2')

In [60]:
x.dtype.names = ('id', 'value', 'note') 

In [61]:
x

array([( 1,  0.5, b'NumPy'), (10, -0.5, b'Essential')],
      dtype=[('id', '<i4'), ('value', '<f4'), ('note', 'S10')])

By assigning the new field names back to the names attribute in the `dtype` object, we can have our customized field names. Or you can do this when you initialize the record arrays by using a list with a tuple, or a dictionary. In the following examples, we are going to create two identical record arrays with customized field names using a list, and a dictionary:

In [77]:
list_x = np.zeros((2,), dtype = [('id', 'i4'), ('value', 'f4', (2,))]) 

In [78]:
list_x

array([(0, [0., 0.]), (0, [0., 0.])],
      dtype=[('id', '<i4'), ('value', '<f4', (2,))])

In [79]:
dict_x = np.zeros((2,), dtype = {'names':['id', 'value'], 'formats':['i4', '2f4']}) 

In [80]:
dict_x

array([(0, [0., 0.]), (0, [0., 0.])],
      dtype=[('id', '<i4'), ('value', '<f4', (2,))])

In the list example, we make a tuple of (field name, data type, and shape) for each field. The shape argument is optional; you may also specify the shape with the data type argument. While using a dictionary to define the field, there are two required keys (`names` and `formats`) and each key has an equally sized list of values.

Before we go on to the next section, we are going to show you how to access multiple fields in your record array all at once. The following example still uses the array `x` that we created at beginning with a customized field: `id`, `value`, and `note`:

In [81]:
x[['id', 'note']]

array([( 1, b'NumPy'), (10, b'Essential')],
      dtype={'names':['id','note'], 'formats':['<i4','S10'], 'offsets':[0,8], 'itemsize':18})

### SQL Like Query on Structured Data

In [85]:
data = np.array([
    ("joe", 32, 6),
    ("mary", 15, 20),
    ("felipe", 80, 100),
    ("beyonce", 38, 9001),
], dtype=[("name", str, 10), ("age", int), ("power", int)])

In [86]:
data[0]

('joe', 32, 6)

In [87]:
data["name"]

array(['joe', 'mary', 'felipe', 'beyonce'], dtype='<U10')

In [92]:
data[data["power"] > 9000]["name"]

array(['joe', 'mary', 'felipe', 'beyonce'], dtype='<U10')

By defining `data`, you create an array, except each item is a `tuple` with a name, an age, and a power level. For the `dtype`, you actually provide a list of tuples with the information about each field: name is a 10-character Unicode field, and both age and power are standard 4-byte or 8-byte integers.

In `data[0]`, you can see that the rows, known as records, are still accessible using the index.

In `data['name']`, you see a new syntax for accessing an entire column, or field.

Finally, in `data[data["power"] > 9000]["name"]`, you see a super-powerful combination of mask-based filtering based on a field and field-based selection. Notice how it’s not that much different to read the following SQL query:

```sql
SELECT name FROM data WHERE power > 9000;
```

In both cases, the result is a list of names where the power level is over 9000.

You can even add in `ORDER BY` functionality by making use of `np.sort()`:

In [73]:
np.sort(data[data["age"] > 20], order="power")["name"]

array(['joe', 'felipe', 'beyonce'], dtype='<U10')

This sorts the data by power before retrieving it, which rounds out your selection of NumPy tools for selecting, filtering, and sorting items just like you might in SQL!

## Dates and time in NumPy

Dates and times are important when you are doing time series analytics, from something as simple as accumulating daily visitors in a museum to something as complicated as trending regression for a crime forecast. Starting from NumPy 1.7, the NumPy core supports date time types. In order to differentiate from the `datetime` object in Python, the data type is called `datetime64`.

This section will cover `numpy.datetime64` creation, time delta arithmetic, and the conversion between units and the Python `datetime`. Let's create a `numpy.datetime64` object by using an ISO string:

In [93]:
x = np.datetime64('2015-04-01') 

In [94]:
y = np.datetime64('2015-04') 

In [95]:
x.dtype

dtype('<M8[D]')

In [77]:
y.dtype

dtype('<M8[M]')

`x` and `y` are both `numpy.datetime64` objects and are constructed from an [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) string.

But the input string for `x` contains a days unit while the string for `y` does not. While creating the NumPy `datetime64`, it will automatically select from the form of the input string, so when we print out the dtype for both `x` and `y`, we can see that `x` with unit `D` stands for days while `y` with unit `M` stands for months. The `<` is also the byteorder, here it is the big-endian, and `M8` is the short notation of `datetime64` (implemented from `np.int64`). The default date units supported by `numpy.datetime64` are years (`Y`), months (`M`), weeks (`W`), and days (`D`), while the time units are hours (`h`), minutes (`m`), seconds (`s`), and milliseconds (`ms`).

Of course we can specify the units when we create the array and also use the `numpy.arange()` method to create the sequence of the array. See the following examples:

In [78]:
y = np.datetime64('2015-04', 'D')

In [83]:
y, y.dtype

(numpy.datetime64('2015-04-01'), dtype('<M8[D]'))

In [84]:
x = np.arange('2015-01', '2015-04', dtype = 'datetime64[M]') 

In [85]:
x

array(['2015-01', '2015-02', '2015-03'], dtype='datetime64[M]')

Next, we are going to do a subtraction of two `numpy.datetime64` arrays, and you will see that the broadcasting rules are still valid as long as the date/time units between two arrays are convertible. We use the same array `x` created earlier and create a new `y` for the following example:

In [88]:
x

array(['2015-01', '2015-02', '2015-03'], dtype='datetime64[M]')

In [89]:
y = np.datetime64('2015-01-01')

In [90]:
x - y

array([ 0, 31, 59], dtype='timedelta64[D]')

Interestingly enough, the result array of `x` subtracting `y` is `[0, 31, 59]`, not the date anymore, and the `dtype` has changed to `timedelta64[D]`. Because NumPy doesn't have a physical quantities system in its core, the `timedelta64` data type was created to complement `datetime64`. In the previous example, `[0, 31, 59]` is the units from `2015-01-01` in each element in `x`, and the units are days (`D`). You may also do the arithmetic between `datetime64` and `timedelta64`, as shown in the following examples:

In [91]:
np.datetime64('2015') + np.timedelta64(12, 'M') 

numpy.datetime64('2016-01')

In [92]:
np.timedelta64(1, 'W') / np.timedelta64(1, 'D') 

7.0

In the last part of this section, we are going to talk about the conversion between `numpy.datetime64` and Python the `datetime`. Although the `datetime64` object inherits many traits from a NumPy array, there are still some benefits from using the Python `datetime` object (such as the `date` and `year` attributes, `isoformat`, and more) or vice versa. For example, you may have a list of `datetime` objects, and you may want to convert them to `numpy.datetime64` for the arithmetic or other NumPy ufuncs. In the following example, we are going to convert the existing `datetime64` array `x` to a list of Python `datetime` in two ways:

In [93]:
x

array(['2015-01', '2015-02', '2015-03'], dtype='datetime64[M]')

In [94]:
x.tolist()

[datetime.date(2015, 1, 1),
 datetime.date(2015, 2, 1),
 datetime.date(2015, 3, 1)]

In [96]:
[element.item() for element in x]

[datetime.date(2015, 1, 1),
 datetime.date(2015, 2, 1),
 datetime.date(2015, 3, 1)]

We can see that `numpy.datetime64.tolist()` and `numpy.datetime64.item()` with the `for` loop can achieve the same goal, that is, converting the array to a list of Python `datetime` objects. But needless to say, we all know which is the more preferred method to do the conversion. On the other hand, if you already have a list of Python datetime and want to convert it to NumPy `datetime64` arrays, simply use the `numpy.array()` function.